<a href="https://colab.research.google.com/github/EyosiyasE/EyosiyasE/blob/main/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q qiskit qiskit-nature==0.7.2 pyscf matplotlib pandas qiskit-aer
!pip install -q pymatgen


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 40.5 MB/s eta 0:00:00


In [3]:
!pip install -q pymatgen

from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

fe_structure = Structure.from_file("/content/FePO4.cif")
li_structure = Structure.from_file("/content/LiFePO4.cif")


li_sites = [site for site in li_structure if site.specie.symbol == "Li"]
li_frac_coords = li_sites[0].frac_coords  # fractional coords of first Li site


mgfe_structure = fe_structure.copy()
mgfe_structure.insert(len(mgfe_structure), "Mg", li_frac_coords)


output_file = "/content/MgFePO4.cif"
mgfe_structure.to(filename=output_file)


spa = SpacegroupAnalyzer(mgfe_structure)
print("Saved as:", output_file)
print("Composition:", mgfe_structure.composition)
print("Space group:", spa.get_space_group_symbol(), spa.get_space_group_number())


Saved as: /content/MgFePO4.cif
Composition: Fe3+4 P5+4 O2-16 Mg1
Space group: P-1 2


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:2945: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Fe0', 'Fe0', 'Fe0', 'Fe0', 'P1', 'P1', 'P1', 'P1', 'O2', 'O2', 'O2', 'O2', 'O2', 'O2', 'O2', 'O2', 'O3', 'O3', 'O3', 'O3', 'O4', 'O4', 'O4', 'O4', 'Mg24']`.
  writer: Any = CifWriter(self, **kwargs)


In [ ]:
import os, math, numpy as np, pandas as pd, matplotlib.pyplot as plt
os.makedirs("results", exist_ok=True)

from pymatgen.core import Structure
from pymatgen.core import Molecule as PMGMolecule
from qiskit_nature.second_q.drivers import PySCFDriver, Molecule
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator
from qiskit.circuit.library import TwoLocal


# The .cif file represents periodic crystals that is infinite in all directions.
# But since Qiskit Nature can only hold a finite number of atoms at once, the following function will cut out a small section

def build_cluster_from_cif(cif_path, cation_symbol=None, fe_o_cut=2.5, p_o_cut=1.75):
  s = Structure.from_file(cif_path)

  fe_indices = [i for i, site in enumerate(s) if site.specie.symbol == "Fe"]
  if not fe_indices:
    raise ValueError("No Fe found in structure.")
  fe_site = s[fe_indices[0]] #This should locate the first Iron in the given .cif structure


  #After finding the iron, which is the redox center, we move towards the oxygen link between Fe and P
  fe_nbrs = s.get_neighbors(fe_site, r=fe_o_cut)
  o_sites = [n.site for n in fe_nbrs if n.site.specie.symbol == "O"]
  if len(o_sites) < 4:
    raise ValueError("Not enough O neighbors found around Fe. Try increasing fe_o_cut.")

  #Then, we move to locate the nearby phosphate group
  o_center = np.mean([o.coords for o in o_sites], axis=0) #Calculate the Geometric Center of Oxygen Neighbors
  p_indices = [i for i, site in enumerate(s) if site.specie.symbol == "P"] #Identify all Phosphorus Atoms
  if not p_indices:
    raise ValueError("No P found in structure.")
  p_index = min(p_indices, key=lambda i: np.linalg.norm(s[i].coords - o_center)) # the minimum is the closest one
  p_site = s[p_index]

  #We move to the oxygen bonded to P atom to locate the phosphae
  p_nbrs = s.get_neighbors(p_site, r=p_o_cut)
  po4_o_sites = [n.site for n in p_nbrs if n.site.specie.symbol == "O"]
  if len(po4_o_sites) < 3:
    p_nbrs = s.get_neighbors(p_site, r=max(1.9, p_o_cut))
    po4_o_sites = [n.site for n in p_nbrs if n.site.specie.symbol == "O"]

  #Finally, this will locate the positive charge cation that sits close to the previously identified structrue and add it to the molecule cluster
  cation_site = None
  if cation_symbol:
    cands = [site for site in s if site.specie.symbol == cation_symbol]
    if cands:
        ref = np.mean([o.coords for o in po4_o_sites], axis=0)
        cation_site = min(cands, key=lambda site: np.linalg.norm(site.coords - ref))

  # So far, the function was able to start from the redox center, Iron, and move towards the cloesest phospate ion and its surrounding cations
  # This helps the Qiskit nature run the simulation with only a representative sample instead of the whole .cif file.

  #This will then gather all the atoms we have gathered in the above codes.
  atoms = []
  seen = set()
  def add(site):
      key = (site.specie.symbol, *np.round(site.coords, 6))
      if key not in seen:
        atoms.append((site.specie.symbol, tuple(site.coords)))
        seen.add(key)


  add(fe_site)
  for o in o_sites: add(o)
  add(p_site)
  for o in po4_o_sites: add(o)
  if cation_site: add(cation_site)

  return atoms # This will now have a list of the locations of each molecules, which I will then use in the next procedure







